In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn import (linear_model, model_selection)
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [12]:
games = pd.read_csv("prepared/combined/games-all.csv")
playerStats = pd.read_csv("prepared/combined/player-stats-all.csv")
table = pd.read_csv("prepared/combined/table-all.csv")
teamStats = pd.read_csv("prepared/combined/team-stats-all.csv")
df = pd.merge(games, table)
df = pd.merge(df, playerStats)
#df = pd.merge(df, teamStats)

In [13]:
y = df["Rk"]

X = df
X.drop(["Rk"], axis=1)
X

,Unnamed: 0,Home,Score,Away,Rk,Squad,W,D,L,GD,Pts,Top Team Scorer,Goalkeeper,Player,MP,Min,Gls,Gls.1
0,1,Avaldsnes,2–1,Vålerenga,2,Avaldsnes,15,3,4,34,48,"Cecilie Pedersen, Elise Thorsnes - 11",Katie Fraine,Meryll Abrahamsen,17,945,1,0.10
1,1,Avaldsnes,2–1,Vålerenga,2,Avaldsnes,15,3,4,34,48,"Cecilie Pedersen, Elise Thorsnes - 11",Katie Fraine,Meryll Abrahamsen,22,1591,5,0.28
2,1,Klepp,1–2,Arna-Bjørnar,2,Avaldsnes,15,3,4,34,48,"Cecilie Pedersen, Elise Thorsnes - 11",Katie Fraine,Meryll Abrahamsen,17,945,1,0.10
3,1,Klepp,1–2,Arna-Bjørnar,2,Avaldsnes,15,3,4,34,48,"Cecilie Pedersen, Elise Thorsnes - 11",Katie Fraine,Meryll Abrahamsen,22,1591,5,0.28
4,1,Sandviken,0–2,Røa,2,Avaldsnes,15,3,4,34,48,"Cecilie Pedersen, Elise Thorsnes - 11",Katie Fraine,Meryll Abrahamsen,17,945,1,0.10
5,1,Sandviken,0–2,Røa,2,Avaldsnes,15,3,4,34,48,"Cecilie Pedersen, Elise Thorsnes - 11",Katie Fraine,Meryll Abrahamsen,22,1591,5,0.28
6,2,Grand Bodø,2–2,Arna-Bjørnar,3,Stabæk,13,4,5,23,43,Melissa Bjånesøy - 11,Ingrid Hjelmseth,Live Aandal,5,315,0,0.00
7,2,Grand Bodø,2–1,Trondheims-Ørn,3,Stabæk,13,4,5,23,43,Melissa Bjånesøy - 11,Ingrid Hjelmseth,Live Aandal,5,315,0,0.00
8,2,Klepp,1–0,Avaldsnes,3,Stabæk,13,4,5,23,43,Melissa Bjånesøy - 11,Ingrid Hjelmseth,Live Aandal,5,315,0,0.00
9,3,Røa,1–2,Stabæk,4,Klepp,12,4,6,10,40,Tameka Butt - 14,Oda Bogstad,Ine Agnethe Aarskog,3,165,0,0.00


In [14]:
X_train, X_valtest, y_train, y_valtest = model_selection.train_test_split(X, y, test_size=0.4,random_state=42)
X_valid, X_test, y_valid, y_test = model_selection.train_test_split(X_valtest, y_valtest, test_size=0.5, random_state=42)

In [15]:
models = {'kNN1': KNeighborsRegressor(n_neighbors=1),
          'kNN10': KNeighborsRegressor(n_neighbors=10),
          'kNN20': KNeighborsRegressor(n_neighbors=20),
          'kNN50': KNeighborsRegressor(n_neighbors=50),
          'LinR': linear_model.LinearRegression(),
          'RFor': RandomForestRegressor(random_state=0)}

bestmodel = None
bestmodelscore = 10
print("RMSE Score:")
for key, model in models.items():
    model = model.fit(X_train, y_train)
    score = mean_squared_error(y_valid, model.predict(X_valid), squared=False)
    if score<bestmodelscore:
        bestmodelscore = score
        bestmodel = model
    print(f"{key}: {score:.5f}")

RMSE Score:


ValueError: could not convert string to float: 'Røa'

In [ ]:
scores = {key: r2_score(y_valid, model.predict(X_valid)) for key, model in models.items()}
scoredf = pd.DataFrame(scores, index=['r2 score']).transpose().sort_values(by='r2 score', ascending=False)
print(scoredf)